# Testing en-forecasting-portoseguro

In [1]:
!pip install datasets peft bitsandbytes safetensors scikit-learn
!pip install torch torchvision torchaudio
!pip install transformers
!pip install loguru
!pip install peft
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate -U


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


### Load Model
Load the base model and finetuned model

In [2]:
from sklearn.metrics import accuracy_score,f1_score
from datasets import load_dataset
from peft import PeftModel
from tqdm import tqdm
import pandas as pd
import datasets
import torch
from transformers import (
    AutoModel,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
    LlamaForCausalLM
)

base_model = "daryl149/llama-2-13b-chat-hf"
peft_model = "/workspace/Llama-3.1-fin-output"

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
model = LlamaForCausalLM.from_pretrained(
    base_model,
    # load_in_8bit=True,
    # load_in_4bit=True,
    trust_remote_code=True,
    device_map='auto',
    torch_dtype=torch.float16,
)
model = PeftModel.from_pretrained(model, peft_model)

model = model.eval()
tokenizer.pad_token = tokenizer.eos_token


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some parameters are on the meta device because they were offloaded to the cpu.
Some parameters are on the meta device because they were offloaded to the cpu.


### Testing code

You can follow one of the files here: https://github.com/AI4Finance-Foundation/FinNLP/tree/main/finnlp/benchmarks

In [4]:

def test(model, tokenizer, batch_size = 6, prompt_fun = None ):
    dataset = load_dataset('TheFinAI/NWGI_test')
    dataset = dataset['test']
    dataset = dataset.to_pandas()
    #dataset = dataset.head(50)

    # print example
    print(f"\n\nPrompt example:\n{dataset['query'][0]}\n\n")

    context = dataset['query'].tolist()
    context = [f"""
    Instruction: What is the sentiment of this news? Please choose an answer from (strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive). \nInput: {x}\nAnswer:""" for x in context]
    total_steps = dataset.shape[0] // batch_size + 1
    print(f"Total len: {len(context)}. Batchsize: {batch_size}. Total steps: {total_steps}")


    out_text_list = []
    for i in tqdm(range(total_steps)):
        tmp_context = context[i* batch_size: (i+1)* batch_size]
        tokens = tokenizer(tmp_context, return_tensors='pt', padding=True)
        for k in tokens.keys():
            tokens[k] = tokens[k].cuda()
        res = model.generate(**tokens, max_length=256)
        res_sentences = [tokenizer.decode(i, skip_special_tokens=True) for i in res]
        print(res_sentences[0])
        out_text = [o.split("Answer:  ")[1] for o in res_sentences]
        print(out_text)
        out_text_list += out_text
        torch.cuda.empty_cache()

    dataset["out_text"] = out_text_list
    # dataset["new_target"] = dataset["target"].apply(change_target)
    # dataset["new_out"] = dataset["out_text"].apply(change_target)
    

    acc = accuracy_score(dataset["answer"], dataset["out_text"])
    f1_macro = f1_score(dataset["answer"], dataset["out_text"], average = "macro")
    f1_micro = f1_score(dataset["answer"], dataset["out_text"], average = "micro")
    f1_weighted = f1_score(dataset["answer"], dataset["out_text"], average = "weighted")

    print(f"Acc: {acc}. F1 macro: {f1_macro}. F1 micro: {f1_micro}. F1 weighted (BloombergGPT): {f1_weighted}. ")
    print(f"Acc: {acc}. ")

    return dataset

dataset = test(model, tokenizer)



Prompt example:
What is the sentiment of this news?
In the latest trading session, Adobe Systems (ADBE) closed at $535.98, marking a +0.31% move from the previous day.
Please choose an answer from {strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive}.


Total len: 4047. Batchsize: 6. Total steps: 675


  0%|          | 1/675 [00:06<1:10:21,  6.26s/it]


    Instruction: What is the sentiment of this news? Please choose an answer from (strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive). 
Input: What is the sentiment of this news?
In the latest trading session, Adobe Systems (ADBE) closed at $535.98, marking a +0.31% move from the previous day.
Please choose an answer from {strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive}.
Answer:  neutral
['neutral', 'strong negative', 'moderately negative', 'strong negative', 'moderately positive', 'moderately negative']


  0%|          | 2/675 [00:12<1:10:30,  6.29s/it]


    Instruction: What is the sentiment of this news? Please choose an answer from (strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive). 
Input: What is the sentiment of this news?
Amazon will use Hawaiian Airlines to fly its first Airbus freighters on primary cargo routes and to Hawaii beginning in the second half of 2023, the airline's top executive said late Friday. The retail giant selected Hawaiian Airlines because of the carrier's familiarity with the A330-300 aircraft, cargo handling experience, track record completing flights and on-time performance as a passenger airline, according to President and CEO Peter Ingram.
Please choose an answer from {strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive}.
Answer:  neutral
['neutral', 'moderately positive', 'strong positive', 'strong negative', 'strong negative', 'strong negative']


...

  5%|▍         | 32/675 [03:20<1:06:22,  6.19s/it]


    Instruction: What is the sentiment of this news? Please choose an answer from (strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive). 
Input: What is the sentiment of this news?
Twenty-four Amazon investors are calling on the tech giant to increase transparency in tax disclosures and adopt a new reporting standard, the Financial Times reported on Sunday.
Please choose an answer from {strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive}.
Answer:  strong negative
['strong negative', 'strong negative', 'neutral', 'strong negative', 'strong negative', 'moderately positive']


  5%|▍         | 33/675 [03:26<1:07:05,  6.27s/it]


    Instruction: What is the sentiment of this news? Please choose an answer from (strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive). 
Input: What is the sentiment of this news?
Starbucks Corp.'s soon-to-be chief executive is a “world-class leader” with deep knowledge of consumer-product companies and will spend time as a barista to get to know the company, the retailer's founder Howard Schultz said Thursday.
Please choose an answer from {strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive}.
Answer:  moderately positive
['moderately positive', 'strong negative', 'moderately positive', 'moderately negative', 'neutral', 'strong negative']


ValueError: Input length of input_ids is 262, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.